---
对于一般的非线性规划问题，由于不是凸规划，就不能使用 cvxpy 库求解。对于一般的非线性规划问题可以使用 `scipy.optimize` 模块的 `minimize` 函数求解

In [1]:
import numpy as np
from scipy.optimize import minimize

c2 = [[-1, -0.15], [-0.15, -2]]
c1 =  [98, 277]
obj = lambda x: x@c2@x + c1@x
cons = [
    {'type': 'ineq', 'fun': lambda x: 100 - x[0] - x[1]},
    {'type': 'ineq', 'fun': lambda x: -x[0] + 2*x[1]},
]
bd = [(0, np.inf) for _ in range(2)]
ret = minimize(obj, np.random.randn(2), constraints=cons, bounds=bd)
print(ret)

     fun: 0.0
     jac: array([ 97.99999998, 276.99999994])
 message: 'Optimization terminated successfully'
    nfev: 7
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([0., 0.])


In [2]:
obj = lambda x: sum(x**2) + 8
def constr1(x):
    x1, x2, x3 = x
    return [
        x1**2 - x2 + x3**2,
        20 - x1 - x2**2 - x3**3,
    ]
def constr2(x):
    x1, x2, x3 = x
    return [
        -x1 - x2**2 + 2,
        x2 + 2*x3**2 - 3,
    ]
cons = [
    {'type': 'ineq', 'fun': constr1},
    {'type': 'eq', 'fun': constr2},
]
bd = [(0, np.inf) for _ in range(3)]
ret = minimize(obj, np.random.randn(3), constraints=cons, bounds=bd)
print(ret)

     fun: 10.651091957236876
     jac: array([1.10433519, 2.40651822, 1.89564812])
 message: 'Optimization terminated successfully'
    nfev: 71
     nit: 14
    njev: 14
  status: 0
 success: True
       x: array([0.55216758, 1.2032591 , 0.94782406])


In [3]:
import cvxpy as cp

c = np.arange(1, 5)
a = np.array([[1,-1,-1,1], [1,-1,1,-3], [1,-1,-2,3]])
b = np.array([0, 1, -1/2])

x = cp.Variable(4)
obj = cp.Minimize(c @ cp.abs(x))
cons = [a @ x == b]
prob = cp.Problem(obj, cons)
prob.solve(solver='GUROBI')
print(f'最优解为：{x.value}'), print(f'最优值为：{prob.value}')

最优解为：[ 0.25  0.    0.   -0.25]
最优值为：1.25


(None, None)

### 供应与选址问题

In [4]:
import pandas as pd
from sklearn.metrics import euclidean_distances

df = pd.read_csv('../05第5章  非线性规划和多目标规划模型/data5_8.txt', header=None, sep='\t')
data = df.values
loc = np.array(data[:2]).T
demand = data[2, :]
reserve = [20, 20]

def obj(xyz):
    pos1, pos2, z = [xyz[:2]], [xyz[2:4]], xyz[4:].reshape(2, 6)
    d = np.vstack([euclidean_distances(pos1, loc), euclidean_distances(pos2, loc)])
    return np.sum(np.multiply(z, d))

def constr1(xyz):
    z = xyz[4:].reshape(2, 6)
    return reserve - np.sum(z, axis=1)

def constr2(xyz):
    z = xyz[4:].reshape(2, 6)
    return np.sum(z, axis=0) - demand

cons = [
    {'type': 'ineq', 'fun': constr1},
    {'type': 'eq', 'fun': constr2},
]
bd = [(-np.inf, np.inf)]*4 + [(0, np.inf)]*12   # x:2, y:2, z:2*6 total(xyz):16
ret = minimize(obj, np.random.randn(16), constraints=cons, bounds=bd)
print(ret)
s = np.round(ret.x, 4)
print('-'*100)
print('目标函数的最优值：', round(ret.fun,4))
print('料场1的坐标为: ', s[:2])
print('料场1到工地的运输量为: \n', s[4:10])
print('料场2的坐标为: ', s[2:4])
print('料场2到工地的运输量为: \n', s[10:])

     fun: 71.93519933765236
     jac: array([-1.04764462e+00,  4.88901234e+00, -3.28159332e-03, -2.07805634e-03,
        8.84590340e+00,  7.15891075e+00,  7.38664341e+00,  4.45112419e+00,
        4.43001175e+00,  0.00000000e+00,  4.42706108e+00,  7.05828381e+00,
        2.79988194e+00,  5.21846771e-01,  1.33450222e+00,  4.73553467e+00])
 message: 'Optimization terminated successfully'
    nfev: 743
     nit: 42
    njev: 42
  status: 0
 success: True
       x: array([7.25000020e+00, 7.74999995e+00, 3.26511196e+00, 5.19197106e+00,
       0.00000000e+00, 5.00000000e+00, 0.00000000e+00, 1.12344928e-14,
       0.00000000e+00, 1.10000000e+01, 3.00000000e+00, 0.00000000e+00,
       4.00000000e+00, 7.00000000e+00, 6.00000000e+00, 0.00000000e+00])
----------------------------------------------------------------------------------------------------
目标函数的最优值： 71.9352
料场1的坐标为:  [7.25 7.75]
料场1到工地的运输量为: 
 [ 0.  5.  0.  0.  0. 11.]
料场2的坐标为:  [3.2651 5.192 ]
料场2到工地的运输量为: 
 [3. 0. 4. 7. 6. 0.]
